# 1.1.4.2: Proyecciones y Mínimos Cuadrados Ordinarios (OLS)

## Objetivos de Aprendizaje

Al completar este notebook, serás capaz de:

- **Interpretar** geométricamente la solución de mínimos cuadrados como la **proyección ortogonal** de un vector sobre un subespacio.
- **Derivar y entender** las **Ecuaciones Normales**: $X^T X \hat{\beta} = X^T \vec{y}$, la base del OLS.
- **Verificar** la condición fundamental de ortogonalidad del vector de error ($X^T \vec{e} = \vec{0}$).
- **Construir** y resolver un modelo de regresión lineal 'desde cero' usando este método, en lugar de funciones de alto nivel.

In [ ]:
# --- Celda de Configuración (Oculta) ---
%display latex
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

--- 
## ⚙️ El Arsenal de Datasets: Nuestra Fuente de Ejercicios

Para este notebook, que se sumerge en la mecánica interna de OLS, usaremos datasets que nos permitan visualizar claramente la geometría de las proyecciones y verificar las propiedades matemáticas de la solución.

In [ ]:
# === CONFIGURACIÓN DE DATASETS ===
from src.data_generation.create_student_performance import create_student_performance_data
from src.data_generation.create_business_data import create_business_data
from src.data_generation.create_geometric_shapes import create_geometric_shapes

# Configuración centralizada de aleatoriedad para REPRODUCIBILIDAD
rng = np.random.default_rng(seed=42)

# === Generación de Datasets y Matrices para este Notebook ===

# 💡 CONTEXTO PEDAGÓGICO: Hilo Conductor (Regresión OLS)
# Usaremos el dataset de estudiantes para implementar la solución de OLS a través de las
# Ecuaciones Normales, verificando que obtenemos el mismo resultado que con `lstsq`.
datos_estudiantes = create_student_performance_data(rng, simplified=True, n_samples=100)

# 💡 CONTEXTO PEDAGÓGICO: Geometría de la Proyección
# Para visualizar la idea de proyectar un vector sobre un plano, crearemos un ejemplo
# simple y de baja dimensión donde podamos graficar todos los elementos: el espacio, el vector y su proyección.
X_plano = create_geometric_shapes(rng, 'line', n_samples=10).values # Generamos una base para un plano
X_plano = np.c_[X_plano, X_plano[:, 0] - X_plano[:, 1]] # Creamos un plano en R^3

# 💡 CONTEXTO PEDAGÓGICO: Regresión Múltiple con Ecuaciones Normales
# Demostraremos que la belleza de las Ecuaciones Normales es que funcionan igual de bien
# para la regresión múltiple, usando el dataset de negocio.
datos_negocio = create_business_data(rng, n_samples=150)

print("Datasets generados y listos para usar.")

## 1. La Geometría de la "Mejor Solución"

En el notebook anterior, aceptamos que `np.linalg.lstsq` nos daba la "mejor solución" a un sistema inconsistente $X\vec{\beta} = \vec{y}$. Pero, ¿qué significa "mejor" geométricamente?

1.  **El Problema:** El vector de nuestros datos reales, $\vec{y}$ (las calificaciones), no se encuentra en el subespacio generado por las columnas de $X$, que llamamos **Espacio Columna** de $X$, o $\text{Col}(X)$. $\text{Col}(X)$ representa el universo de todas las posibles predicciones lineales que podemos hacer.

2.  **La Solución:** Si no podemos alcanzar $\vec{y}$ exactamente, la "mejor solución" es el vector $\hat{y}$ (o $\vec{p}$) *dentro* de $\text{Col}(X)$ que está **más cerca** de $\vec{y}$. Este punto es la **proyección ortogonal** de $\vec{y}$ sobre el subespacio $\text{Col}(X)$.

**La Gran Intuición:** La distancia más corta de un punto (la punta de $\vec{y}$) a un plano ($\	ext{Col}(X)$) se logra con una línea perpendicular. Esto significa que el **vector de error**, $\vec{e} = \vec{y} - \hat{y}$, debe ser **ortogonal** a *todo* el espacio columna de $X$.

### Ejemplo Demostrativo 1: Visualizando la Proyección Ortogonal

In [ ]:
# 1. DATOS: Creamos un "plano" (espacio columna de X) y un vector "y" que no está en el plano.
X = np.array([[1, 0, 1], [0, 1, 1], [1, 1, 2], [1, -1, 0]]) # X es 4x3 pero tiene rango 2
y = np.array([4, 5, 10, 2])

# 2. APLICACIÓN: Encontramos la proyección de y sobre el espacio columna de X usando lstsq.
beta_hat, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
y_hat = X @ beta_hat # Esta es la proyección 'p'
error_vec = y - y_hat

# 3. INTERPRETACIÓN Y VERIFICACIÓN
print(f"Vector y (original): {y}")
print(f"Vector ŷ (proyección): {np.round(y_hat, 2)}")
print(f"Vector de error (e): {np.round(error_vec, 2)}")

# Verificamos que el error es ortogonal a las columnas de X
orto_check = X.T @ error_vec
print(f"\nVerificación de Ortogonalidad (X.T @ e): {np.round(orto_check, 9)}")
print("Los valores son prácticamente cero, confirmando la ortogonalidad.")

# (La visualización 3D/4D es compleja, pero el principio numérico se mantiene)

## 2. La Solución Algebraica: Las Ecuaciones Normales

La condición geométrica de que el error $\vec{e}$ es ortogonal al espacio columna de $X$ nos lleva directamente a una solución algebraica elegante.

#### Derivación Detallada
1.  La condición de ortogonalidad significa que el producto punto de $\vec{e}$ con **cada columna** de $X$ es cero. 
2.  Podemos escribir esto de forma compacta como: $ X^T \vec{e} = \vec{0} $.
3.  Sustituimos la definición del error, $\vec{e} = \vec{y} - \hat{y}$. Y como $\hat{y} = X\hat{\beta}$, tenemos $\vec{e} = \vec{y} - X\hat{\beta}$.
    $$ X^T (\vec{y} - X\hat{\beta}) = \vec{0} $$
4.  Finalmente, distribuimos $X^T$ y reordenamos para obtener las **Ecuaciones Normales**:
    $$ X^T X \hat{\beta} = X^T \vec{y} $$

¡Este es un sistema **cuadrado y simétrico** que podemos resolver para $\hat{\beta}$! La matriz $X^T X$ a veces se llama la "matriz de Gram". Si las columnas de $X$ son linealmente independientes, $X^T X$ es invertible.

### Ejemplo Demostrativo 2: Resolviendo OLS con las Ecuaciones Normales

In [ ]:
# 1. DATOS: Usamos nuestro Hilo Conductor.
X_feature = datos_estudiantes[['horas_estudio']].values
X = np.c_[np.ones(X_feature.shape[0]), X_feature]
y = datos_estudiantes['calificacion_examen'].values

# 2. CONSTRUIR LAS ECUACIONES NORMALES
# Lado izquierdo de la ecuación
XTX = X.T @ X
# Lado derecho de la ecuación
XTy = X.T @ y

print(f"Matriz XᵀX (2x2):")
print(np.round(XTX, 2))
print(f"\nVector Xᵀy (2x1):")
print(np.round(XTy, 2))

# 3. RESOLVER EL SISTEMA CUADRADO: (XᵀX)β = (Xᵀy)
# Como XᵀX es cuadrada, ahora podemos usar np.linalg.solve()
beta_hat = np.linalg.solve(XTX, XTy)

# 4. INTERPRETACIÓN
print(f"\nCoeficientes β_hat resueltos: {np.round(beta_hat, 2)}")

# Comparamos con el resultado de lstsq del notebook anterior
beta_lstsq, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
print(f"Coeficientes β_hat con lstsq: {np.round(beta_lstsq, 2)}")
print(f"¿Son iguales? {np.allclose(beta_hat, beta_lstsq)}")

---
## 4. Ejercicios Guiados con Scaffolding (8+)
Rellena las partes marcadas con `# COMPLETAR` para afianzar tu comprensión.

### === EJERCICIO GUIADO 1: Construir los Componentes de las Ecuaciones Normales ===

In [ ]:
# DATOS: Una matriz de diseño X y un vector y simples.
X = np.array([[1, 2], [1, 3], [1, 5]])
y = np.array([3, 4, 6])

# TODO 1: Calcula la matriz de Gram, XᵀX.
XTX = # COMPLETAR

# TODO 2: Calcula el vector del lado derecho, Xᵀy.
XTy = # COMPLETAR

# VERIFICACIÓN
XTX_esperada = np.array([[3, 10], [10, 38]])
XTy_esperada = np.array([13, 47])
assert np.allclose(XTX, XTX_esperada)
assert np.allclose(XTy, XTy_esperada)
print("✅ ¡Componentes de las Ecuaciones Normales calculados correctamente!")
print(f"XᵀX =\n{XTX}")
print(f"\nXᵀy = {XTy}")

### === EJERCICIO GUIADO 2: Resolver las Ecuaciones Normales ===

In [ ]:
# DATOS: Los componentes XᵀX y Xᵀy del ejercicio anterior.
XTX = np.array([[3, 10], [10, 38]])
XTy = np.array([13, 47])

# TODO: Resuelve el sistema cuadrado (XᵀX)β = (Xᵀy) para encontrar beta_hat.
# PISTA: Usa np.linalg.solve().
beta_hat = # COMPLETAR

# VERIFICACIÓN
assert beta_hat.shape == (2,)
assert np.allclose(beta_hat, np.array([1.5, 0.85714]))
print("✅ ¡Sistema de Ecuaciones Normales resuelto correctamente!")
print(f"β_hat = {np.round(beta_hat, 2)}")

### === EJERCICIO GUIADO 3: Calcular el Vector de Proyección y el Error ===

In [ ]:
# DATOS: La matriz X original y el beta_hat que acabamos de encontrar.
X = np.array([[1, 2], [1, 3], [1, 5]])
y = np.array([3, 4, 6])
beta_hat = np.linalg.solve(X.T @ X, X.T @ y)

# TODO 1: Calcula el vector de predicciones y_hat (la proyección de y sobre Col(X)).
y_hat = # COMPLETAR

# TODO 2: Calcula el vector de error (residuos).
error = # COMPLETAR

# VERIFICACIÓN
assert y_hat.shape == (3,)
assert error.shape == (3,)
print("✅ ¡Cálculos correctos!")
print(f"y_hat (proyección) = {np.round(y_hat, 2)}")
print(f"error (residuos) = {np.round(error, 2)}")

### === EJERCICIO GUIADO 4: Verificar la Ortogonalidad del Error ===

In [ ]:
# DATOS: La X, y_hat y error del ejercicio anterior.
X = np.array([[1, 2], [1, 3], [1, 5]])
y = np.array([3, 4, 6])
y_hat = X @ np.linalg.solve(X.T @ X, X.T @ y)
error = y - y_hat

# El vector de error debe ser ortogonal a CADA columna de X.

# TODO 1: Calcula el producto punto del error con la primera columna de X.
dot_col1 = # COMPLETAR

# TODO 2: Calcula el producto punto del error con la segunda columna de X.
dot_col2 = # COMPLETAR

# VERIFICACIÓN
assert np.isclose(dot_col1, 0) and np.isclose(dot_col2, 0)
# Una forma más compacta de verificar es X.T @ error, que debe ser un vector de ceros.
assert np.allclose(X.T @ error, np.zeros(2))
print("✅ ¡Verificación de ortogonalidad exitosa!")
print("El vector de error es perpendicular al espacio de las predicciones.")

--- 
# 5. Banco de Ejercicios Prácticos (30+)
Ahora te toca a ti. Resuelve estos ejercicios para consolidar tu conocimiento.

### Parte A: Componentes de las Ecuaciones Normales

**A1 (🟢 Fácil):** Dados $A = \begin{pmatrix} 1 & 1 \\ 1 & 2 \\ 1 & 3 \end{pmatrix}$ y $\vec{b} = \begin{pmatrix} 1 \\ 2 \\ 2 \end{pmatrix}$, calcula a mano (y luego verifica con código) $A^T A$ y $A^T \vec{b}$.

**A2 (🟢 Fácil):** Usando el `datos_estudiantes`, construye la matriz de diseño $X$ y el vector $y$. Calcula $X^T X$.

**A3 (🟡 Medio):** ¿Por qué la matriz $X^T X$ es siempre cuadrada, incluso si $X$ es una matriz alta y delgada?

**A4 (🟡 Medio):** Usando `datos_negocio`, construye la matriz de diseño $X$ para predecir `ventas_mensuales` a partir de `precio` y `gasto_marketing`. Calcula $X^T X$. ¿De qué tamaño es?

### Parte B: Resolución y Verificación

**B1 (🟢 Fácil):** Usando los resultados de A1, resuelve las Ecuaciones Normales para encontrar $\hat{x}$.

**B2 (🟡 Medio):** Para los datos de A1, calcula el vector de proyección $\vec{p}=A\hat{x}$ y el vector de error $\vec{e}=\vec{b}-\vec{p}$.

**B3 (🟡 Medio):** Verifica que el vector de error $\vec{e}$ del ejercicio B2 es ortogonal a las columnas de A.

**B4 (🔴 Reto):** Implementa desde cero la regresión múltiple para el problema de `datos_negocio` (ejercicio A4). Resuelve las Ecuaciones Normales para encontrar los 3 coeficientes beta. Compara tu resultado con `np.linalg.lstsq`.

### Parte C: Geometría y Aplicaciones

**C1 (🟡 Medio):** ¿Qué le pasaría a la matriz $X^T X$ si las columnas de $X$ fueran perfectamente ortogonales? ¿Cómo simplificaría esto la solución de las Ecuaciones Normales?

**C2 (🔴 Reto):** La matriz de Proyección que mapea cualquier vector $\vec{y}$ a su proyección sobre el espacio columna de $X$ es $P = X(X^T X)^{-1} X^T$. Usando los datos de A1, calcula esta matriz $P$. Verifica que $P\vec{b}$ te da el mismo vector de proyección $\vec{p}$ que calculaste en B2.

**C3 (🔴 Reto):** Una propiedad de las matrices de proyección es que son idempotentes ($P^2 = P$). Verifica que la matriz $P$ que calculaste en C2 cumple esta propiedad.

---

## ✅ Mini-Quiz de Autoevaluación

1. ¿Cuál es la fórmula de las Ecuaciones Normales?
2. ¿Cuál es la relación geométrica entre el vector de error $\vec{e}$ y el espacio columna de la matriz $X$ en una regresión OLS?
3. Si las columnas de $X$ son linealmente dependientes (multicolinealidad), ¿qué problema ocurre con la matriz $X^T X$ que impide resolver las Ecuaciones Normales?
4. La solución de mínimos cuadrados, $\hat{\beta}$, produce un vector de predicciones $\hat{y} = X\hat{\beta}$. Este vector $\hat{y}$ es la ___________ de $\vec{y}$ sobre el espacio columna de $X$.

## 🚀 Próximos Pasos

¡Felicidades! Has entendido la bella geometría que hace funcionar la regresión lineal, derivando y aplicando su solución desde los primeros principios.

- Aunque las Ecuaciones Normales son conceptualmente clave, pueden ser numéricamente inestables si $X^T X$ está mal condicionada (alto número de condición). En el **próximo notebook** exploraremos métodos numéricamente más robustos para resolver problemas de mínimos cuadrados, como la **descomposición QR**.